# 02 Data cleaning 

#### Python Requirements
Code written in Python 3.7, libraries required includes pandas and os only 

## Index Links
- [Import and clean zipcodes/counties related datasets](#Import-an-clean-zipcodes/counties-related-datasets )
- [Centroids of zipcodes latitude longitude](#Centroids-of-zipcodes-latitude-longitude)
- [State related datasets](#State-related-datasets )
- [Merging DataFrames with common zipcodes](#Merging-DataFrames-with-common-zipcodes)

In [33]:
import pandas as pd

## Import and clean zipcodes/counties related datasets 
---------------

In [34]:
df_incomes = pd.read_csv('./Data/zip_income.csv')
df_incomes = df_incomes.drop('Unnamed: 0', axis=1)
df_incomes.rename(columns = {'income':'tot_income'}, inplace = True)
list(df_incomes.columns)

['zipcode', 'tot_income', 'state']

In [35]:
df_pops = pd.read_csv('./Data/zip_population.csv')
df_pops = df_pops[['zip_code', 'y-2016']]
df_pops.rename(columns={'zip_code':'zipcode','y-2016':'population'},inplace = True)
list(df_pops.columns)

['zipcode', 'population']

In [36]:
df_longlat = pd.read_csv('./Data/zip_cities.csv')
df_longlat = df_longlat[['Zipcode', 'City', 'Lat', 'Long']]
df_longlat = df_longlat.dropna()
df_longlat.rename(index=str, columns={'Zipcode': 'zipcode', 'Long': 'long', 'Lat': 'lat','City':'city'}, inplace=True)
list(df_longlat.columns)

['zipcode', 'city', 'lat', 'long']

In [37]:
df_counties = pd.read_csv('./Data/zip_counties.csv')
df_counties.drop(df_counties.columns[2:],axis=1,inplace = True)
df_counties.columns = ['zipcode','county']
df_counties['county'] = df_counties['county'].str.replace(' County','')
list(df_counties.columns)

['zipcode', 'county']

## Getting positional centroids of all zipcodes 
-------------------

We commented this part in order to not insert 340Mb file in our github download. For reference : https://github.com/OpenDataDE/State-zip-code-GeoJSON

This initial file returned a boundarie for each zipcode under the form of a large aray. We chose to only keep the centroids (middle position) of the boundaries instead of this huge array. 

In [38]:
import os

In [39]:
#extensions = [f for f in os.listdir('./State-zip-code-GeoJSON-master/') if f.endswith('.json')]
#extension_names = [f[:2].upper() for f in os.listdir('./State-zip-code-GeoJSON-master/') if f.endswith('.json')]

In [40]:
def getting_centroids(extension,extension_name):
    dataframe = pd.read_json('./State-zip-code-GeoJSON-master/'+extension)
    centroids = []
    for index in range(dataframe.shape[0]) : 
        zipcode = dataframe['features'][index]['properties']['ZCTA5CE10']
        lat = dataframe['features'][index]['properties']['INTPTLAT10']
        lon = dataframe['features'][index]['properties']['INTPTLON10']
        centroids.append([zipcode,lat,lon,extension_name])

    return centroids

In [41]:
#flat_centroids = []

#for sublist in [getting_centroids(extensions[i],extension_names[i]) for i in range(len(extensions))]:
    #for item in sublist:
        #flat_centroids.append(item)

#df_centroids = pd.DataFrame(flat_centroids,columns = ['zipcode','lat','lon','state'])
#df_centroids['zipcode'] = pd.to_numeric(df_centroids['zipcode'])
#df_centroids['lat'] = pd.to_numeric(df_centroids['lat'])
#df_centroids['lon'] = pd.to_numeric(df_centroids['lon'])

## Getting positional centroids of all counties
---------------------

In [42]:
extensions = [f for f in os.listdir('./Data/USA/')]

In [43]:
def getting_data(extensions):
    to_go_for = './Data/USA/' + extensions
    files_name  = [f for f in os.listdir(to_go_for)]
    counties = []
    names = []
    for file_name in files_name : 
        file = dict(pd.read_json(to_go_for + '/' + file_name,typ='series'))
        counties.append(file['features'][0])
        names.append(extensions)
    return counties,names

In [44]:
names = [getting_data(i)[1] for i in extensions]
dictionaries = [getting_data(i)[0] for i in extensions]

In [45]:
flat_names = []
for sublist in names:
    for item in sublist:
        flat_names.append(item)

flat_dictionaries = []
for sublist in dictionaries:
    for item in sublist:
        flat_dictionaries.append(item)


In [46]:
df_counties_centroids_temp  = pd.DataFrame([flat_names,flat_dictionaries])
df_counties_centroids = df_counties_centroids_temp.T.copy()
df_counties_centroids.columns = ['states','dictionaries']

## Import state industry statistics datasets 
----------------------

State data dictionary 

In [47]:
df_state_professions_description = pd.read_excel('./Data/occupation_description.xlsx')
df_state_professions_description.drop([i for i in range(8)],inplace = True)
df_state_professions_description.drop([41,42,43,44,45,46],inplace =True)
df_state_professions_description.rename(columns = {'May 2017 OES Estimates':'acronym','Unnamed: 1':'description'},inplace = True)
df_state_professions_description.set_index('acronym',inplace = True)

In [48]:
df_state_professions = pd.read_excel('./Data/state_occupations.xlsx')
df_state_professions.drop(['ANNUAL','HOURLY','STATE'],axis =1,inplace = True)
df_state_professions_ram = df_state_professions[(df_state_professions['OCC_GROUP'] == 'major')|(df_state_professions['OCC_GROUP'] == 'total')].copy()
df_state_professions_ram = df_state_professions_ram[['OCC_TITLE','ST','TOT_EMP','JOBS_1000','H_MEAN','A_MEAN','A_PCT10','A_PCT25','A_MEDIAN','A_PCT75','A_PCT90']]
df_state_professions_ram['OCC_TITLE'] =df_state_professions_ram['OCC_TITLE'].str.replace(' Occupations', '')
df_state_professions_ram.columns = ['occupation','state','tot_employement','perc','h_mean','annual_mean','10','25','med','75','90']

## Merging DataFrames with common zipcodes 
----------------------

In [59]:
df_centroids = pd.read_csv('./centroids.csv')

In [60]:
set_inc = set(df_incomes['zipcode'])
set_pop = set(df_pops['zipcode'])
set_cou = set(df_counties['zipcode'])
set_lon = set(df_longlat['zipcode'])
set_cen = set(df_centroids['zipcode'])

In [61]:
in_pops_and_incomes = [i for i in set_inc if i in set_pop]
in_pops_incomes_and_counties = [i for i in set_cou if i in in_pops_and_incomes]
in_all = [i for i in set_lon if i in in_pops_incomes_and_counties]

In [62]:
first = pd.merge(df_incomes[df_incomes['zipcode'].isin(in_all)],df_pops[df_pops['zipcode'].isin(in_all)],on ='zipcode')
second = pd.merge(first,df_longlat[df_longlat['zipcode'].isin(in_all)],on = 'zipcode')

In [63]:
df_complete = pd.merge(second, df_counties[df_counties['zipcode'].isin(in_all)],on = 'zipcode')
df_complete = df_complete.merge(df_centroids,on='zipcode')

In [64]:
df_complete.drop(['lat_x','long','state_y'],axis =1,inplace = True)
df_complete.columns = ['zipcode','tot_income','state','population','city','county','lat','lon']

## Exporting our 4 clean datasets 
------------------

In [65]:
df_complete.drop(['state','lat','lon'],axis = 1).head()

,zipcode,tot_income,population,city,county
0,35004,290260,10418,MOODY,St. Clair
1,35005,121690,7708,ADAMSVILLE,Jefferson
2,35006,57418,3099,ADGER,Jefferson
3,35006,57418,3099,ADGER,Tuscaloosa
4,35006,57418,3099,ADGER,Walker


In [66]:
df_state_professions_ram[df_state_professions_ram['state'] == 'AL']

,occupation,state,tot_employement,perc,h_mean,annual_mean,10,25,med,75,90
0,All,AL,1922570,1000,20.76,43170,17770,21740,32800,52020,78690
1,Management,AL,69950,36.385,53.44,111150,52130,70130,96980,133360,188860
34,Business and Financial Operations,AL,73110,38.028,34.33,71410,35870,47780,64350,86530,111310
63,Computer and Mathematical,AL,39760,20.68,39.36,81870,42150,56630,77590,102020,128500
79,Architecture and Engineering,AL,43010,22.37,42.41,88220,43400,60530,83660,113840,141880
112,"Life, Physical, and Social Science",AL,8260,4.295,31.11,64700,33570,42870,56330,78040,105730
144,Community and Social Service,AL,17980,9.352,21.11,43920,25450,32960,42040,54670,64110
161,Legal,AL,10080,5.245,41.86,87080,31140,43410,64360,114170,168640
170,"Education, Training, and Library",AL,103290,53.724,22.87,47560,17880,27480,46290,58020,69650
227,"Arts, Design, Entertainment, Sports, and Media",AL,17440,9.07,21.55,44820,18200,24090,35930,54630,78980


In [68]:
df_complete.to_csv('complete.csv',index = False)
df_state_professions_ram.to_csv('industries.csv',index = False)
df_centroids.to_csv('centroids.csv',index = False)
df_counties_centroids.to_csv('counties.csv',index = False)

In [ ]:
from sklearn.externals import joblib
def test(x):
    x += 2
    return x

joblib.dump(test,'test.csv') 
test_2 = joblib.load('test.csv')
test_2(4)